In [247]:
import os
import json
from ultralytics import YOLO
from PIL import Image

# 加载 YOLO 模型
model = YOLO("YOLOv8x_Symbols.pt")

# 设置图像路径
source_dir = "./samples/"
image_files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# 定义用于 MIDI 的标签集合
note_classes = {
    'noteheadBlack', 'noteheadHalf', 'noteheadWhole',
    'restQuarter', 'restHalf', 'restWhole',
    'noteheadBlackOnLine', 'noteheadBlackInSpace',
    'noteheadHalfOnLine', 'noteheadWholeInSpace', 'rest8th'
}
# staff 也需要保留，用于 pitch 匹配
extra_classes = {'staff'}

# 推理图像
results = model(image_files)

# 结果保存路径
save_json = "./samples/midi_notes.json"
output = []

for result in results:
    img_path = result.path
    boxes = result.boxes.data
    confs = result.boxes.conf
    cls_ids = result.boxes.cls

    notes = []
    for box, conf, cls_id in zip(boxes, confs, cls_ids):
        label = result.names[int(cls_id)]
        # if label not in note_classes and label not in extra_classes:
        #     continue  # 不是音符也不是五线谱，跳过

        x1, y1, x2, y2 = map(float, box[:4])
        note_data = {
            "label": label,
            "confidence": float(conf),
            "bbox": [x1, y1, x2, y2],
            "center": [(x1 + x2) / 2, (y1 + y2) / 2]
        }
        notes.append(note_data)

    output.append({
        "filename": os.path.basename(img_path),
        "notes": notes
    })

# 保存 JSON
with open(save_json, 'w') as f:
    json.dump(output, f, indent=2)

print(f"🎼 Done! MIDI-relevant predictions saved to {save_json}")



0: 224x992 1 brace, 1 clefG, 1 clefF, 34 noteheadBlackOnLines, 43 noteheadBlackInSpaces, 9 beams, 2 staffs, 348.8ms
Speed: 1.3ms preprocess, 348.8ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 992)
🎼 Done! MIDI-relevant predictions saved to ./samples/midi_notes.json


In [248]:
import os
import json
from PIL import Image, ImageDraw, ImageFont

def visualize_predictions(json_path, image_dir, save_dir):
    # 读取json
    with open(json_path, 'r') as f:
        data = json.load(f)

    os.makedirs(save_dir, exist_ok=True)

    for page in data:
        filename = page["filename"]
        image_path = os.path.join(image_dir, filename)

        if not os.path.exists(image_path):
            print(f"❗ Image {filename} not found!")
            continue

        img = Image.open(image_path).convert("RGB")
        draw = ImageDraw.Draw(img)

        try:
            font = ImageFont.truetype("arial.ttf", size=20)
        except:
            font = ImageFont.load_default()

        for note in page["notes"]:
            label = note["label"]
            conf = note["confidence"]
            bbox = note["bbox"]
            center = note["center"]

            x1, y1, x2, y2 = bbox
            cx, cy = center

            # 颜色区分
            if 'staff' in label:
                color = (0, 0, 255)  # 蓝色
            elif 'notehead' in label:
                color = (0, 255, 0)  # 绿色
            elif 'rest' in label:
                color = (255, 0, 0)  # 红色
            else:
                color = (255, 165, 0)  # 橙色其他

            # 画框
            draw.rectangle([(x1, y1), (x2, y2)], outline=color, width=2)
            # 写label
            draw.text((x1, y1-20), f"{label} {conf:.2f}", fill=color, font=font)
            # 标记center
            r = 3
            draw.ellipse([(cx-r, cy-r), (cx+r, cy+r)], fill=color)

        save_path = os.path.join(save_dir, filename)
        img.save(save_path)
        print(f"✅ Saved visualization: {save_path}")

# 使用
visualize_predictions(
    json_path="./samples/midi_notes.json",
    image_dir="./samples/",
    save_dir="./samples/visualized/"
)


✅ Saved visualization: ./samples/visualized/1016b70de8eca72d3684dde722d1201.png


In [249]:
import json
import os
from PIL import Image, ImageDraw

def visualize_staffs(json_path, image_dir, save_dir="./samples/staff_visualization"):
    os.makedirs(save_dir, exist_ok=True)

    with open(json_path, 'r') as f:
        data = json.load(f)

    for page in data:
        filename = page["filename"]
        notes = page["notes"]
        staff_notes = [n for n in notes if n["label"] == "staff"]

        # 打开图像
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path).convert("RGB")
        draw = ImageDraw.Draw(image)

        print(f"\n🖼 Image: {filename}")
        for i, s in enumerate(sorted(staff_notes, key=lambda n: n["center"][1])):
            x1, y1, x2, y2 = s["bbox"]
            cx, cy = s["center"]
            draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
            draw.line([(x1, cy), (x2, cy)], fill="blue", width=1)
            draw.text((x1, y1 - 10), f"staff {i+1}", fill="green")
            print(f"staff {i+1}: top={y1:.1f}, bottom={y2:.1f}, center={cy:.1f}")

        out_path = os.path.join(save_dir, f"staff_{filename}")
        image.save(out_path)
        print(f"✅ Saved visual to {out_path}")

# 示例使用
visualize_staffs(
    json_path="./samples/midi_notes.json",
    image_dir="./samples"
)


🖼 Image: 1016b70de8eca72d3684dde722d1201.png
staff 1: top=78.6, bottom=148.7, center=113.6
staff 2: top=261.7, bottom=330.9, center=296.3
✅ Saved visual to ./samples/staff_visualization\staff_1016b70de8eca72d3684dde722d1201.png


In [250]:
import json
import mido
from mido import Message, MidiFile, MidiTrack
import os
import math
def c_major_transform_clefG(i):
    dic = {
        6: 81,
        5: 79,
        4: 77,
        3: 76,
        2: 74,
        1: 72,
        0: 71,
        -1: 69,
        -2: 67,
        -3: 65,
        -4: 64,
        -5: 62,
        -6: 60
    }
    return dic.get(i, 60)
def c_major_transform_clefF(i):
    dic = {
        6: 60,
        5: 59,
        4: 57,
        3: 55,
        2: 53,
        1: 52,
        0: 50,
        -1: 48,
        -2: 47,
        -3: 45,
        -4: 43,
        -5: 41,
        -6: 40
    }
    return dic.get(i, 60)
def get_note_name(midi_note):
    """Returns the note name for a MIDI note number (e.g., 60 → C4)"""
    note_names = {
        0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E', 5: 'F',
        6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'
    }
    octave = midi_note // 12 - 1  # MIDI note 60 is C4
    note = midi_note % 12
    return f"{note_names[note]}{octave}"
def get_duration_from_symbol(symbol_type, has_beam=False):
    """
    Returns the MIDI duration in ticks for a given note or rest type
    
    Args:
        symbol_type: The label of the symbol (e.g., 'noteheadBlack')
        has_beam: Whether the note has a beam attached, indicating eighth note
    """
    if "Whole" in symbol_type:
        return 1920  # 4 beats * 480 ticks
    elif "Half" in symbol_type:
        return 960   # 2 beats * 480 ticks
    elif "Black" in symbol_type and has_beam:
        return 240   # 0.5 beat * 480 ticks (eighth note)
    elif "Black" in symbol_type:
        return 480   # 1 beat * 480 ticks (quarter note)
    elif "8th" in symbol_type or "Eighth" in symbol_type:
        return 240   # 0.5 beat * 480 ticks
    elif "16th" in symbol_type:
        return 120   # 0.25 beat * 480 ticks
    else:
        # Default to quarter note
        return 480

def assign_elements_to_staves(elements, staves):
    """
    Assign elements (notes, beams, flags, etc.) to their appropriate staff
    
    Args:
        elements: List of elements with bbox and center coordinates
        staves: List of staff elements with bbox coordinates
    
    Returns:
        Dictionary mapping staff indices to lists of elements belonging to that staff
    """
    # Sort staves by vertical position (top to bottom)
    sorted_staves = sorted(staves, key=lambda x: x.get("center", [0, 0])[1])
    
    # Initialize result dictionary
    staff_elements = {i: [] for i in range(len(sorted_staves))}
    
    # For each element, find the closest staff
    for element in elements:
        element_y = element.get("center", [0, 0])[1]
        
        # Find the closest staff by vertical distance
        closest_staff_idx = None
        min_distance = float('inf')
        
        for i, staff in enumerate(sorted_staves):
            staff_y = staff.get("center", [0, 0])[1]
            distance = abs(element_y - staff_y)
            
            if distance < min_distance:
                min_distance = distance
                closest_staff_idx = i
        
        # Assign the element to the closest staff
        if closest_staff_idx is not None:
            staff_elements[closest_staff_idx].append(element)
    
    return staff_elements

def is_note_beamed(note, beams):
    """
    Check if a note is connected to a beam within the same staff
    
    Args:
        note: The note object with bbox and center
        beams: List of beam objects with bbox and center for the same staff
    
    Returns:
        Boolean indicating if the note is connected to a beam
    """
    note_x = note.get("center", [0, 0])[0]
    note_bbox = note.get("bbox", [0, 0, 0, 0])
    threshold = note_bbox[3] - note_bbox[1]
    # Only check horizontal alignment with each beam (vertical is handled by staff assignment)
    for beam in beams:
        beam_bbox = beam.get("bbox", [0, 0, 0, 0])
        beam_x1, beam_y1, beam_x2, beam_y2 = beam_bbox
        
        # Check if note's x-coordinate is within the beam's x-range
        if beam_x1 - threshold <= note_x <= beam_x2 + threshold:  # Small tolerance
            return True
    
    return False

def is_note_flagged(note, flags):
    """
    Check if a note has a flag (8th, 16th, etc.) within the same staff
    
    Args:
        note: The note object with bbox and center
        flags: List of flag objects with bbox and center for the same staff
    
    Returns:
        Boolean indicating if the note has a flag
    """
    note_x = note.get("center", [0, 0])[0]
    note_bbox = note.get("bbox", [0, 0, 0, 0])
    threshold = note_bbox[3] - note_bbox[1]
    
    # Check proximity to each flag (only horizontal alignment matters)
    for flag in flags:
        flag_x = flag.get("center", [0, 0])[0]
        
        # Check horizontal proximity
        if abs(flag_x - note_x) < threshold:
            return True
    
    return False

def json_to_midi(json_path, midi_path):
    # Read the JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Create a new MIDI file with standard resolution
    mid = MidiFile(ticks_per_beat=480)
    
    # Create tracks based on detected staves
    tracks = {}
    track_info = {}  # Store clef and other info for each track
    
    # Process each page in the JSON data
    for page in data:
        filename = page.get("filename", "unknown")
        notes = page.get("notes", [])
        
        print(f"Processing {filename} with {len(notes)} elements")
        
        # Extract all elements by type
        staves = [note for note in notes if note.get("label") == "staff"]
        beams = [note for note in notes if note.get("label") == "beam"]
        flags = [note for note in notes if "flag" in note.get("label", "").lower()]
        clefs = [note for note in notes if note.get("label") in ["clefG", "clefF", "clefC"]]
        time_sigs = [note for note in notes if note.get("label", "").startswith("timeSig")]
        
        # Assign elements to staves
        staff_beams = assign_elements_to_staves(beams, staves)
        staff_flags = assign_elements_to_staves(flags, staves)
        staff_clefs = assign_elements_to_staves(clefs, staves)
        
        print(f"Found {len(staves)} staves, {len(beams)} beams, and {len(flags)} flags")
        
        # Sort staves by vertical position (top to bottom)
        sorted_staves = sorted(staves, key=lambda x: x.get("center", [0, 0])[1])
        
        # Create tracks for each staff
        for i, staff in enumerate(sorted_staves):
            staff_id = f"staff_{i}"
            staff_center_y = staff.get("center", [0, 0])[1]
            
            # Get clefs for this staff
            staff_clef_candidates = staff_clefs.get(i, [])
            staff_clef = None
            if staff_clef_candidates:
                # Use the first clef found for this staff
                staff_clef = staff_clef_candidates[0].get("label")
            
            # Create track if it doesn't exist
            if staff_id not in tracks:
                tracks[staff_id] = MidiTrack()
                mid.tracks.append(tracks[staff_id])
                # Add track name
                tracks[staff_id].append(mido.MetaMessage('track_name', name=f"Staff {i+1}"))
                # Default to piano sound (General MIDI program 0)
                tracks[staff_id].append(mido.Message('program_change', program=0, time=0))
                
                # Store staff information
                track_info[staff_id] = {
                    "clef": staff_clef,
                    "staff_y": staff_center_y,
                    "bbox": staff.get("bbox"),
                    "time_signature": (4, 4)  # Default 4/4 time
                }
                
                print(f"Created track for staff {i+1} with clef: {staff_clef}")
    
        # Assign all musical elements to staves
        all_musical_elements = [note for note in notes if "notehead" in note.get("label", "") or "rest" in note.get("label", "")]
        staff_musical_elements = assign_elements_to_staves(all_musical_elements, staves)
        
        # Process musical elements for each staff
        for staff_idx, staff_elements in staff_musical_elements.items():
            staff_id = f"staff_{staff_idx}"
            if staff_id not in track_info:
                continue  # Skip if we don't have track info
                
            staff_data = track_info[staff_id]
            
            # Get beams and flags for this staff
            this_staff_beams = staff_beams.get(staff_idx, [])
            this_staff_flags = staff_flags.get(staff_idx, [])
            
            # Filter notes and rests
            notes_and_rests = []
            for element in staff_elements:
                element_label = element.get("label", "")
                
                # Check if the element is a note or a rest
                is_note = "notehead" in element_label
                is_rest = "rest" in element_label
                
                if is_note or is_rest:
                    # Check if the note is beamed or flagged (for eighth notes)
                    if is_note and "Black" in element_label:
                        has_beam = is_note_beamed(element, this_staff_beams)
                        has_flag = is_note_flagged(element, this_staff_flags)
                        
                        # Store this information with the note
                        element['has_beam'] = has_beam
                        element['has_flag'] = has_flag
                        
                        if has_beam:
                            print(f"  Note at ({element.get('center', [0, 0])[0]:.1f}, {element.get('center', [0, 0])[1]:.1f}) is beamed")
                        if has_flag:
                            print(f"  Note at ({element.get('center', [0, 0])[0]:.1f}, {element.get('center', [0, 0])[1]:.1f}) has a flag")
                    
                    notes_and_rests.append(element)
            
            # Sort by horizontal position (left to right)
            notes_and_rests.sort(key=lambda x: x.get("center", [0, 0])[0])
            
            # Group into temporal events (chords or single notes/rests)
            horizontal_tolerance = 10  # Pixels of tolerance for horizontal alignment
            
            # Group symbols into temporal segments
            temporal_groups = []
            current_group = []
            
            for symbol in notes_and_rests:
                x_pos = symbol.get("center", [0, 0])[0]
                
                if not current_group:
                    # First symbol in a group
                    current_group.append(symbol)
                elif abs(x_pos - current_group[0].get("center", [0, 0])[0]) <= horizontal_tolerance:
                    # Symbol horizontally aligned with current group
                    current_group.append(symbol)
                else:
                    # New horizontal position, start a new group
                    temporal_groups.append(current_group)
                    current_group = [symbol]
            
            # Add the last group if it exists
            if current_group:
                temporal_groups.append(current_group)
            
            print(f"Processing {len(notes_and_rests)} symbols in {len(temporal_groups)} groups for {staff_id}")
            
            # Process each group (chord, single note, or rest)
            current_time = 0  # Accumulated time in ticks
            
            for group in temporal_groups:
                # Determine if this group contains notes, rests, or both
                notes_in_group = [s for s in group if "notehead" in s.get("label", "")]
                rests_in_group = [s for s in group if "rest" in s.get("label", "")]
                
                # If the group contains both notes and rests, prioritize notes
                if notes_in_group:
                    # This is a chord or a single note
                    # Sort notes by vertical position (low to high)
                    notes_in_group.sort(key=lambda x: x.get("center", [0, 0])[1], reverse=True)
                    
                    # Determine duration based on the first note in the group
                    note_type = notes_in_group[0].get("label", "")
                    has_beam = notes_in_group[0].get("has_beam", False)
                    has_flag = notes_in_group[0].get("has_flag", False)
                    duration = get_duration_from_symbol(note_type, has_beam or has_flag)
                    
                    # Process each note in the chord
                    chord_pitches = []
                    for note in notes_in_group:
                        note_center_y = note.get("center", [0, 0])[1]
                        
                        # Calculate pitch based on vertical position relative to staff
                        staff_bbox = staff_data["bbox"]
                        line_height = (staff_bbox[3] - staff_bbox[1]) / 4  # 5 lines = 4 spaces
                        distance_from_center = (staff_data["staff_y"] - note_center_y) / (line_height/2)
                        
                        # Round to nearest staff position
                        staff_position = round(distance_from_center)
                        
                        # Determine pitch based on clef
                        if staff_data["clef"] == "clefG":
                            pitch = c_major_transform_clefG(staff_position)
                        elif staff_data["clef"] == "clefF":
                            pitch = c_major_transform_clefF(staff_position)
                        else:
                            # Default handling for unknown clef
                            pitch = 60 + staff_position  # Middle C + offset
                        
                        # Skip if note is outside reasonable MIDI range
                        if 0 <= pitch <= 127:
                            chord_pitches.append(pitch)
                            duration_type = "quarter note"
                            if has_beam or has_flag:
                                duration_type = "eighth note"
                            print(f"  Added note: {get_note_name(pitch)} (MIDI {pitch}) ({duration_type})")
                    
                    # Play all notes in the chord simultaneously
                    velocity = 64  # Default medium velocity
                    
                    # Add note_on events with the accumulated time for the first note
                    first_note = True
                    for pitch in chord_pitches:
                        if first_note:
                            tracks[staff_id].append(Message('note_on', note=pitch, velocity=velocity, time=current_time))
                            first_note = False
                        else:
                            tracks[staff_id].append(Message('note_on', note=pitch, velocity=velocity, time=0))
                    
                    # Reset current_time since we've used it
                    current_time = 0
                    
                    # Add note_off events
                    # First note_off has the full duration, others immediately follow
                    for i, pitch in enumerate(chord_pitches):
                        if i == 0:
                            tracks[staff_id].append(Message('note_off', note=pitch, velocity=0, time=duration))
                        else:
                            tracks[staff_id].append(Message('note_off', note=pitch, velocity=0, time=0))
                    
                elif rests_in_group:
                    # This is a rest - just add a time delay
                    rest_type = rests_in_group[0].get("label", "")
                    rest_duration = get_duration_from_symbol(rest_type)
                    
                    print(f"  Added rest: {rest_type} with duration {rest_duration}")
                    
                    # Accumulate time for the rest
                    current_time += rest_duration
    
    # Check if tracks are empty and create a default one if needed
    if not tracks:
        print("No valid tracks found, creating default track")
        default_track = MidiTrack()
        mid.tracks.append(default_track)
        default_track.append(mido.MetaMessage('track_name', name="Default Track"))
        default_track.append(mido.Message('program_change', program=0, time=0))
    
    # Save the MIDI file
    mid.save(midi_path)
    print(f"MIDI file saved to {midi_path}")
    return True

In [251]:
json_to_midi(
    json_path="./samples/midi_notes.json",
    midi_path="./samples/output_pitch.mid"
)

Processing 1016b70de8eca72d3684dde722d1201.png with 91 elements
Found 2 staves, 9 beams, and 0 flags
Created track for staff 1 with clef: clefG
Created track for staff 2 with clef: clefF
  Note at (1106.0, 122.4) is beamed
  Note at (804.5, 87.7) is beamed
  Note at (1758.7, 139.8) is beamed
  Note at (1681.4, 122.5) is beamed
  Note at (1797.4, 131.2) is beamed
  Note at (1720.3, 131.3) is beamed
  Note at (234.8, 131.2) is beamed
  Note at (491.3, 131.1) is beamed
  Note at (452.5, 122.4) is beamed
  Note at (607.6, 105.0) is beamed
  Note at (1874.7, 113.9) is beamed
  Note at (646.3, 96.3) is beamed
  Note at (529.8, 122.5) is beamed
  Note at (930.1, 87.6) is beamed
  Note at (684.9, 87.5) is beamed
  Note at (1836.3, 105.2) is beamed
  Note at (965.8, 96.6) is beamed
  Note at (804.6, 105.3) is beamed
  Note at (1913.4, 105.3) is beamed
  Note at (839.3, 78.8) is beamed
  Note at (202.5, 122.5) is beamed
  Note at (1418.9, 105.0) is beamed
  Note at (839.3, 96.5) is beamed
  Note

True

In [252]:
import pygame
import time

midi_file = "./samples/output_pitch.mid"
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(midi_file)
pygame.mixer.music.play()

while pygame.mixer.music.get_busy():
    time.sleep(0.5)
